In [1]:

!pip install -q transformers datasets accelerate scikit-learn pandas emoji

import pandas as pd
import numpy as np
import re
import emoji
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 17.1 MB/s eta 0:00:00


In [2]:
CSV_PATH = "/content/train_CG.csv"
TEXT_COL = "Text"
LABEL_COL = "Label"

# smaller, faster model
MODEL_NAME = "distilbert-base-multilingual-cased"

# speed-oriented settings
MAX_LEN = 64        # shorter sequences
BATCH_SIZE = 8      # smaller batch for less GPU memory
NUM_EPOCHS = 3      # fewer epochs
LR = 3e-5           # a bit higher, to converge in fewer epochs



In [5]:

df = pd.read_csv(CSV_PATH)

print(df.head())
print(df[LABEL_COL].value_counts())


                                                Text              Label
0      Santhu nu masth miss malthondu ullar bro 🤔🤔🤔🤔         uninvolved
1   Undhe n YouTube d thooyere andala mobile bodathe  discouraging hope
2  ತಟ್ಟಿ ನುಂಗೆಲ್ ತಾರೆದ ಮಡಲ್‌ನ್ ನೀರ್‌ಡ್ ಪಾಡ್ದ್ ಅವು...         uninvolved
3                            Super comedy deepak rai   encouraging hope
4                Namma tulundadu comedy first eppodu         uninvolved
Label
uninvolved           2490
encouraging hope     1895
blended hope          895
discouraging hope     711
Name: count, dtype: int64


In [6]:
import emoji
import re

url_pattern = re.compile(r"http\S+|www\.\S+")
user_pattern = re.compile(r"@\w+")
space_pattern = re.compile(r"\s+")

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.strip()
    text = url_pattern.sub("<URL>", text)
    text = user_pattern.sub("@USER", text)

    text = emoji.replace_emoji(text, replace=" ")
    text = space_pattern.sub(" ", text)
    return text

df["clean_text"] = df[TEXT_COL].astype(str).apply(clean_text)


In [7]:
# Standardize label strings (if needed)
df[LABEL_COL] = df[LABEL_COL].str.lower().str.strip()

# Map labels to ids
label_encoder = LabelEncoder()
df["label_id"] = label_encoder.fit_transform(df[LABEL_COL])

id2label = {i: l for i, l in enumerate(label_encoder.classes_)}
label2id = {l: i for i, l in id2label.items()}
num_labels = len(id2label)
print(id2label)

# Train/validation split
train_df, val_df = train_test_split(
    df,
    test_size=0.1,
    random_state=42,
    stratify=df["label_id"]
)

print(len(train_df), len(val_df))


{0: 'blended hope', 1: 'discouraging hope', 2: 'encouraging hope', 3: 'uninvolved'}
5391 600


In [8]:

df[LABEL_COL] = df[LABEL_COL].str.lower().str.strip()


label_encoder = LabelEncoder()
df["label_id"] = label_encoder.fit_transform(df[LABEL_COL])

id2label = {i: l for i, l in enumerate(label_encoder.classes_)}
label2id = {l: i for i, l in id2label.items()}
num_labels = len(id2label)
print(id2label)


train_df, val_df = train_test_split(
    df,
    test_size=0.1,
    random_state=42,
    stratify=df["label_id"]
)

print(len(train_df), len(val_df))


{0: 'blended hope', 1: 'discouraging hope', 2: 'encouraging hope', 3: 'uninvolved'}
5391 600


In [9]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class TuluHopeDataset(Dataset):
    def __init__(self, df, text_col, label_col_id):
        self.texts = df[text_col].tolist()
        self.labels = df[label_col_id].tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        enc = tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=MAX_LEN,
            return_tensors="pt"
        )

        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(label, dtype=torch.long)
        return item

train_dataset = TuluHopeDataset(train_df, "clean_text", "label_id")
val_dataset   = TuluHopeDataset(val_df,   "clean_text", "label_id")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/tulu_hope_ckpt",
    do_train=True,
    do_eval=True,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    logging_steps=50,
    logging_dir="/content/logs",
    report_to="none",
    fp16=True  # Enable mixed-precision training for faster GPU performance
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # Changed 'tokenizer' to 'processing_class' to address FutureWarning
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()

Step,Training Loss
50,1.246600
100,1.156900
150,1.104400
200,1.059200
250,1.011700
300,0.978000
350,0.964500
400,0.970700
450,0.941100
500,0.935100


{'eval_loss': 0.9826560616493225,
 'eval_accuracy': 0.67,
 'eval_f1_macro': 0.5434846741274696,
 'eval_precision': 0.5749454168595673,
 'eval_recall': 0.5395076227817124,
 'eval_runtime': 0.9396,
 'eval_samples_per_second': 638.597,
 'eval_steps_per_second': 79.825,
 'epoch': 3.0}

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    accuracy  = accuracy_score(labels, predictions)
    f1_macro  = f1_score(labels, predictions, average='macro')
    precision = precision_score(labels, predictions, average='macro')
    recall    = recall_score(labels, predictions, average='macro')

    return {
        "accuracy": accuracy,
        "f1_macro": f1_macro,
        "precision": precision,
        "recall": recall
    }